In [ ]:
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import geojson
from geojson import MultiLineString
from shapely import geometry
from shapely.geometry import Point, Polygon, box
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay

In [ ]:
def is_geohash_in_bounding_box(current_geohash, bbox_coordinates):
    """Checks if the box of a geohash is inside the bounding box

    :param current_geohash: a geohash
    :param bbox_coordinates: bounding box coordinates
    :return: true if the the center of the geohash is in the bounding box
    """

    coordinates = geohash.decode(current_geohash)
    geohash_in_bounding_box = (bbox_coordinates[0] < coordinates[0] < bbox_coordinates[2]) and (
            bbox_coordinates[1] < coordinates[1] < bbox_coordinates[3])
    return geohash_in_bounding_box
def build_geohash_box(current_geohash):
    """Returns a GeoJSON Polygon for a given geohash

    :param current_geohash: a geohash
    :return: a list representation of th polygon
    """

    b = geohash.bbox(current_geohash)
    polygon = [(b['w'], b['s']), (b['w'], b['n']), (b['e'], b['n']), (b['e'], b['s'],), (b['w'], b['s'])]
    return polygon
def compute_geohash_tiles(bbox_coordinates,GEOHASH_PRECISION):
    """Computes all geohash tile in the given bounding box

    :param bbox_coordinates: the bounding box coordinates of the geohashes
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding box, assuming the earth is flat ;)
    center_latitude = (bbox_coordinates[0] + bbox_coordinates[2]) / 2
    center_longitude = (bbox_coordinates[1] + bbox_coordinates[3]) / 2

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            if neighbor not in checked_geohashes and is_geohash_in_bounding_box(neighbor, bbox_coordinates):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes
def create_polygon(coordinates):
    polygon_geom = Polygon(coordinates)
    crs = {'init': 'epsg:4326'}
    polygon = gp.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])     
    return polygon.geometry # return polygon['geometry']
# this function is for save the geohashes into a json file 
def write_geohash_layer(geohashes,filename):
    """Writes a grid layer based on the geohashes

    :param geohashes: a list of geohashes
    """

    layer = MultiLineString([build_geohash_box(gh) for gh in geohashes])
    with open('result/' + filename + '.json', 'wb') as f:
        f.write(geojson.dumps(layer,sort_keys=True).encode('utf-8'))

def compute_geohash_tiles_from_polygon(polygon,GEOHASH_PRECISION):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes

In [107]:
msia_gdf = gp.GeoDataFrame.from_file('msia_routes_kl/msia_routes_kl.shp', geometry='geometry')

In [4]:
len(msia_gdf)

In [5]:
msia_gdf.head()

In [6]:
msia_poi = gp.GeoDataFrame.from_file('msia_poi_kl/msia_poi_kl.shp', geometry='geometry')

In [12]:
msia_poi.head()
msia_poi.dtypes

osm_id       object
code          int64
fclass       object
name         object
GID_0        object
NAME_0       object
GID_1        object
NAME_1       object
NL_NAME_1    object
GID_2        object
NAME_2       object
VARNAME_2    object
NL_NAME_2    object
TYPE_2       object
ENGTYPE_2    object
CC_2         object
HASC_2       object
geometry     object
dtype: object

In [37]:
#print(msia_poi['geometry'].iloc[0][0].x)
#print(msia_poi['geometry'])
msia_poi['lat'] = msia_poi['geometry'].centroid.y
msia_poi['lon'] = msia_poi['geometry'].centroid.x

In [123]:
msia_poi.head(100)

,osm_id,code,fclass,name,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,...,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,lat,lon,geohash,cx_count,cx_count_int
0,158731933,2501,supermarket,Hero,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5458488 3.0876767)),3.087677,101.545849,'w281wdp',True,500
1,158732943,2504,mall,Ole Ole,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5169616 3.0434654)),3.043465,101.516962,'w281q44',True,500
2,158741219,2302,fast_food,McDonald's,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5475146 3.0403635)),3.040364,101.547515,'w281qc2',False,100
3,168928764,2110,hospital,SALAM Shah Alam Specialist Hospital,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5353548 3.0491463)),3.049146,101.535355,'w281q7p',True,500
4,168931187,2301,restaurant,Seri Bunga,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5386002 3.0510745)),3.051074,101.538600,'w281qe6',False,100
5,168932609,2601,bank,Maybank,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5420034 3.0558818)),3.055882,101.542003,'w281qsk',True,500
6,168935492,2101,pharmacy,Farmasi Mesra Maju,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5422999 3.0553058)),3.055306,101.542300,'w281qsh',False,100
7,168936696,2301,restaurant,Restoran Panmour Villa,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5415213 3.0567032)),3.056703,101.541521,'w281qsk',False,100
8,168937382,2301,restaurant,Ali's Corner,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5415622 3.056312)),3.056312,101.541562,'w281qsk',False,100
9,168938322,2301,restaurant,Pizza Hut,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.7_1,...,Daerah,District,None,MY.SL.PE,(POINT (101.5418317 3.0564068)),3.056407,101.541832,'w281qsk',False,100


In [40]:
### Create bbox msia

In [41]:
bbox_coordinates = [2.442250, 101.026847, 3.853692, 101.874907]
concentratedarea = compute_geohash_tiles(bbox_coordinates,7)
print(len(concentratedarea))

634276


In [42]:
# STEP 2 
# CONVERT THE COORDINATES INTO POLYGON
myshape = pd.DataFrame(concentratedarea)
myshape.columns = ['geohash']
# # tak yah buat di bawah pun tak ngapa

In [43]:
concentratedarea[1]

'w281ut1'

In [73]:
myshape.head()


,geohash
0,w281ut4
1,w281ut1
2,w281ut5
3,w281usf
4,w281usc


In [69]:
msia_poi['geohash'] = msia_poi['geometry'].apply(lambda x:compute_geohash_tiles_from_polygon(x,7))

In [46]:
len(msia_poi)

5377

In [57]:
#len(msia_poi.fclass.unique())
#len(msia_poi.code.unique())
#msia_poi.fclass.unique()
import numpy as np
tier1 = ['atm', 'post_office', 'public_building', 
         'supermarket', 'bank', 'mall', 'school',
         'town_hall', 'hospital', 'university', 'cafe', 'convenience']

msia_poi['cx_count'] = msia_poi['fclass'].isin(tier1)
msia_poi['cx_count_int'] = np.where(msia_poi['cx_count']==True, 500, 100 )


In [71]:
msia_poi['geohash'] = msia_poi['geohash'].astype(str)
msia_poi['geohash'] = msia_poi['geohash'].str[1:]
msia_poi['geohash'] = msia_poi['geohash'].str[:-1]

In [72]:
msia_poi['geohash'].head()

0    'w281wdp'
1    'w281q44'
2    'w281qc2'
3    'w281q7p'
4    'w281qe6'
Name: geohash, dtype: object

In [109]:
msia_route = msia_gdf
msia_route.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,4931450,5122,residential,Jalan Pinggiran USJ 1/17,None,B,0,0,F,F,...,None,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5552027 3.0501994, 101.5562486..."
1,4931468,5115,tertiary,Jalan Pinggiran USJ 1,None,B,0,0,F,F,...,None,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5548825 3.0500569, 101.5550899..."
2,4931834,5122,residential,Persiaran Perusahaan U/1,None,B,0,0,F,F,...,None,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5546656 3.091444, 101.5546525 ..."
3,4931877,5113,primary,Jalan Montford,B9,F,0,0,F,F,...,None,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5533042 3.0878122, 101.5527241..."
4,4934921,5113,primary,Bulatan Megawati,None,F,0,0,F,F,...,None,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5435172 3.0521068, 101.5433673..."


In [110]:
msia_route['geohash'] = msia_route['geometry'].apply(lambda x:compute_geohash_tiles_from_polygon(x,7))

In [111]:
msia_route['geohash'] = msia_route['geohash'].astype(str)
msia_route['geohash'] = msia_route['geohash'].str[1:]
msia_route['geohash'] = msia_route['geohash'].str[:-1]

In [112]:
msia_route.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,geohash
0,4931450,5122,residential,Jalan Pinggiran USJ 1/17,None,B,0,0,F,F,...,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5552027 3.0501994, 101.5562486...",'w281r52'
1,4931468,5115,tertiary,Jalan Pinggiran USJ 1,None,B,0,0,F,F,...,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5548825 3.0500569, 101.5550899...",'w281r50'
2,4931834,5122,residential,Persiaran Perusahaan U/1,None,B,0,0,F,F,...,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5546656 3.091444, 101.5546525 ...",'w281wfj'
3,4931877,5113,primary,Jalan Montford,B9,F,0,0,F,F,...,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5533042 3.0878122, 101.5527241...",'w281wfh'
4,4934921,5113,primary,Bulatan Megawati,None,F,0,0,F,F,...,MYS.15.7_1,Petaling,Shah Alam,None,Daerah,District,None,MY.SL.PE,"LINESTRING (101.5435172 3.0521068, 101.5433673...",'w281qet'


In [113]:
merge = msia_route.merge(msia_poi, on='geohash', how='left')

In [121]:

merge['cx_count_int'] = merge['cx_count_int'].fillna(0)
merge['cx_count_int'].unique()

array([  0., 100., 500.])

In [114]:
merge.head()
merge.columns
merge = merge.drop(['geometry_y', 'cx_count'], axis=1)
#merge.drop

In [116]:
merge.columns
len(merge)

179825

In [122]:
merge = GeoDataFrame(merge, geometry='geometry_x')
#merge_road.head()
merge.to_file('msia_route_weight/')

In [74]:
msia_poi.groupby('fclass').count()[['osm_id']]

,osm_id
fclass,
artwork,8
atm,202
attraction,38
bakery,36
bank,312
bar,9
beauty_shop,28
bench,13
beverages,22


In [47]:
msia_poi.fclass.unique()

array(['supermarket', 'mall', 'fast_food', 'hospital', 'restaurant',
       'bank', 'pharmacy', 'convenience', 'school', 'police',
       'vending_parking', 'department_store', 'viewpoint', 'toilet',
       'theatre', 'tower', 'picnic_site', 'public_building', 'food_court',
       'water_tower', 'cafe', 'fire_station', 'playground', 'bakery',
       'post_box', 'post_office', 'car_dealership', 'clothes',
       'furniture_shop', 'prison', 'community_centre', 'hotel',
       'golf_course', 'attraction', 'sports_centre', 'kiosk', 'car_wash',
       'laundry', 'college', 'university', 'greengrocer', 'doctors',
       'gift_shop', 'cinema', 'courthouse', 'atm', 'chemist', 'dentist',
       'bookshop', 'telephone', 'toy_shop', 'guesthouse', 'recycling',
       'graveyard', 'vending_machine', 'hostel', 'camp_site',
       'tourist_info', 'bicycle_rental', 'observation_tower', 'pub',
       'bar', 'fountain', 'museum', 'town_hall', 'outdoor_shop',
       'kindergarten', 'butcher', 'swimming_p

In [125]:
msia_poi[['lat', 'lon', 'cx_count_int']].to_csv('heatmap.csv')